
<img src="img/viu_logo.png" width="200">

## 01MIAR - Procesamiento de Datos

![logo](img/python_logo.png)

*Ivan Fuertes / Franklin Alvarez*

In [ ]:
import numpy as np
import pandas as pd

# Duarante esta sesión mostraremos números flotantes con una precisión de 2 decimales
pd.set_option('display.precision', 2)

### Uniendo datasets con 'join' y 'merge'
- merge() == join()
 - 'join' utiliza por defecto los índices para unir
- Utilizando el parámetro 'on'
 - Si las columnas difieren, 'left_on' y 'right_on'
 
https://miro.medium.com/v2/resize:fit:1400/1*GigXPhr4Ue2zbrgIIoB8Lw.png

#### Combinar varios datasets 
- En base a un elemento en común (índice)
- MovieLens 'UserId'

In [ ]:
import zipfile as zp # para descomprimir archivos zip
import urllib.request # para descargar de URL
import os

# descargar MovieLens dataset
url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'  
local_zip = os.path.join("res", "ml-1m.zip")
urllib.request.urlretrieve(url, local_zip)

# descomprimiendo archivo zip
with zp.ZipFile(local_zip, 'r') as zipp: 
    print('Extracting all files...') 
    zipp.extractall(os.path.join("res")) # destino
    print('Done!') 

In [ ]:
ruta_users = os.path.join("res", "ml-1m", "users.dat")
ruta_ratings = os.path.join("res", "ml-1m", "ratings.dat")
ruta_movies = os.path.join("res", "ml-1m", "movies.dat")

users_dataset = pd.read_csv(ruta_users, sep='::', index_col=0,
    header=None, names=['UserID','Gender','Age','Occupation','Zip-code'], engine='python', encoding="ISO-8859-1")

ratings_dataset = pd.read_csv(ruta_ratings, sep='::', index_col=0, 
    header=None, names=['UserID','MovieID','Rating','Timestamp'], engine='python', encoding="ISO-8859-1")

movies_dataset = pd.read_csv(ruta_movies, sep='::', index_col=0, 
    header=None, names=['MovieID','Title','Genre'], engine='python', encoding="ISO-8859-1")

In [ ]:
display(users_dataset.head(5))
print(len(users_dataset))

In [ ]:
display(ratings_dataset.sample(5))
print(len(ratings_dataset))

In [ ]:
display(movies_dataset.sample(5))
print(len(movies_dataset))

In [ ]:
# combinando users y ratings, ¿Cómo?
combined_dataset = users_dataset.merge(ratings_dataset, on='UserID', how='inner') # parametro 'on' define la columna pivote
display(combined_dataset.head(5))
print(len(combined_dataset))

In [ ]:
# combinando movies y el resto
all_dataset = combined_dataset.merge(movies_dataset, on='MovieID', how='inner')
display(all_dataset.sample(5))
print(len(combined_dataset))

#### Concatenate
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

## Pivot
- Representar los datos en función a varios parámetros, agregando
```python
pivot_table(<lista de valores>, index=<agregador primario>, columns=<agregador secundario>)
```
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

In [ ]:
display(all_dataset.pivot_table('Rating', index='Gender', columns='Age'))

# Operación equivalente para cada celda
mask = (all_dataset['Gender'] == 'F') * (all_dataset['Age'] == 1)
F1 = all_dataset[mask].Rating.mean()
print("Valor para F-1:", F1)

In [ ]:
# Podemos agregar funciones o lista de funciones con las que operar
display(all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc='count'))
# display(all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc=['count', 'max', 'mean']))

## Agrupaciones
- agg -> funciones estadísticas de agregación
- Series.unique() -> valores únicos
- pd.value_counts -> ocurrencias

In [ ]:
valores_unicos = all_dataset[mask].Rating.unique()
print(valores_unicos)

In [ ]:
ocurrencias = all_dataset[mask].Rating.value_counts()
ocurrencias.name = "Número de ocurrencias por cada Rating"
print(ocurrencias)

## Manipulación de strings
```python
split(): separar en bloques en función de un carácter
replace(): reemplazar un carácter por otro
index(): encontrar la posición de un carácter
```

### Ejemplo MovieLens: Separar géneros y año en columnas individuales

In [ ]:
# Ejemplo con MovieLens: Genre
## 1: obtener todos los géneros por separado
## 2: crear un dataset de géneros
## 3: por película, marcar género por separado
## 4: Extraer el año de cada peléicula y colocar en columna individual
## 5: Remover el año del título
## 6: unir con dataset genre
display(movies_dataset.head(3))

In [ ]:
all_genres = movies_dataset['Genre'].apply(lambda x : x.split('|'))
print(all_genres)

In [ ]:
genres = {genre for x in all_genres for genre in x}
print("Set con todos los géneros:")
print(genres)
print()

# Alternativa usando unique()
genres = pd.unique(pd.Series(all_genres.sum()))
print("Lista con todos los géneros:")
print(genres)

In [ ]:
# crear tabla con columnas por género
zeros = np.zeros( (len(movies_dataset), len(genres)) )
genres_frame = pd.DataFrame(zeros, columns=genres, index=list(range(1, len(movies_dataset) + 1)))
display(genres_frame.head(3))

In [ ]:
columns_genres = genres_frame.columns # lista de generos (columnas)

# para cada película, marcar género con 1
for i, genre in enumerate(movies_dataset['Genre']):
    inds = columns_genres.get_indexer(genre.split('|')) # retorna los indices correspondientes a los generos de cada pelicula
    genres_frame.iloc[i,inds] = 1 # localiza las columnas del genero correspondiente, marca con 1

display(genres_frame.head(5))

#### Replace e index para extraer el año de la película

In [ ]:
display(movies_dataset.head(2))

In [ ]:
# extraer el año de la columna Title
def split_year(title):
    index = title.index('(')
    return title[index:].replace('(','').replace(')','')
    
# crear nueva columna Year
movies_dataset['Year'] = movies_dataset['Title'].apply(split_year)
display(movies_dataset.head(2))

In [ ]:
# eliminar el año de la columna Title
def remove_year(title):
    index = title.index('(')
    return title[:index-1].rstrip()

movies_dataset['Title'] = movies_dataset['Title'].apply(remove_year)
display(movies_dataset.head(2))

In [ ]:
# unir con dataset original
movies_split_genre = movies_dataset.join(genres_frame)

display(movies_split_genre.head(5))

In [ ]:
# Remover vieja columna de genre
movies_split_genre.drop('Genre', axis=1, inplace=True)
display(movies_split_genre.head(5))

#### Ejemplo con Regular Expressions
Algunos métodos str aceptan expresiones regulares cómo parámetros. Por ejemplo `match()` y `extract()`
- https://docs.python.org/3/library/re.html
- https://regex101.com/
- import re

##### Ejemplo con `match`: Eliminar zip-code con formato erróneo 

In [ ]:
# ¿Cómo localizar que 'Zip-code' tiene un formato erróneo?
users_dataset.sample(5)

In [ ]:
# Regular expression: ^\d{5}$
# ^ = start of the string
# \d = decimal string
# {5} = 5 repeticiones de decimales
# $ = end of string

mask_correcto = users_dataset['Zip-code'].str.match('^\\d{5}$') # Formato correcto
mask_incorrecto = mask_correcto == False
print("Existen {} entradas con el formato incorrecto".format(mask_incorrecto.sum()))

display(users_dataset[mask_incorrecto])

In [ ]:
index_incorrecto = users_dataset[mask_incorrecto].index
print(index_incorrecto)
users_dataset_clean = users_dataset.drop(users_dataset[mask_incorrecto].index)
print("Tamaño del dataset antes de la limpieza:", users_dataset.shape)
print("Tamaño del dataset después de la limpieza:", users_dataset_clean.shape)

##### Ejemplo con `extract`: Extraer año

In [ ]:
movies_dataset = pd.read_csv(ruta_movies, sep='::', index_col=0, 
    header=None, names=['MovieID','Title','Genre'], engine='python', encoding="ISO-8859-1")
display(movies_dataset.head(2))

In [ ]:
# ¿Cómo extraer el año con regex en el formato adecuado?
# Regular expression: (\d{4})
# (= busca apertura parentesis
# \d = decimal string
# {4} = 4 repeticiones de decimales
# ) = cierre de parentesis

movies_year = movies_dataset['Title'].str.extract('(\\d{4})')
movies_dataset['Year'] = movies_year
movies_dataset['Title'] = movies_dataset['Title'].apply(remove_year)

display(movies_dataset.head(2))

## Ejercicio de operaciones con colecciones: Filtrar películas de 1975
```python
reduce: aplicar una operación y retornar un valor
map: aplicar  una operación y retornar una secuencia
filter: retorna una secuencia con elementos que cumplen una condición
```


#### Reduce
- Aplicar una operación matemática a cada uno de los elementos de una colección
- Diferente de 'apply()' porque retorna un valor numérico
- Ejemplo: Detección de géneros en años específicos

https://docs.python.org/3/library/functools.html

In [ ]:
from functools import reduce # necesario para reduce

lista = [1, 3, 5, 7, 9]
print(reduce(lambda x,y: x + y, lista))

In [ ]:
movies_1975 = movies_split_genre[ movies_split_genre['Year'].str.contains('1975') ]
display(movies_1975.head(4))

In [ ]:
any_drama = reduce(lambda x,y : bool(x) | bool(y), movies_1975['Animation']) # hay algún drama en 1975
print(any_drama)

all_comedy = reduce(lambda x,y : bool(x) & bool(y),movies_1975['Comedy']) # son todas las películas de 1975 comedias?
print(all_comedy)

In [ ]:
print(movies_1975['Drama'].any()) # Comprueba si hay algún valor que puede cumplir  
print(movies_1975['Comedy'].all()) # Comprueba si todos los valores son True

In [ ]:
# Observar el tipo de dato antes para ver si es posible aplicar las funciones
print(movies_1975.dtypes)
print()
print("Sólo 2 valores únicos para cualquier género:", movies_1975['Comedy'].unique())

#### Map
- Aplicar  una operación (lambda) y retornar una secuencia
- Verifica que géneros aparecen en las películas de 1975
- Elimina las columnas de los géneros que no aparecen

In [ ]:
mapa = map(lambda x : (x, movies_1975[x].any()), movies_1975)
print(mapa) # Objeto no realizado con los resultados de la operación lambda

display(pd.Series(mapa)) # Normalmente se realiza con pd.Series

In [ ]:
mapa = map(lambda x : movies_1975[x].any(), movies_1975)

columnas_validas = movies_1975.columns[pd.Series(mapa)]
movies_1975_clean = movies_1975[columnas_validas]
display(movies_1975_clean.head(4))

#### Filter
- retorna una secuencia con elementos que cumplen una condición
- Ejemplo: obtener las películas de 1975 que contienen 'The' en el título

In [ ]:
filtro = filter(lambda x : 'The' in x, movies_1975_clean['Title'])

for movie in filtro:
    print(movie)
# ¿Están todos los títulos con "The"? si tiene mayúsculas o no...

In [ ]:
filtro = filter(lambda x : 'the' in x, movies_1975['Title'].str.lower())
list(filtro)

## Cambiar tipo de datos

In [ ]:
movies_1975_bool = pd.DataFrame()
for col in movies_1975_clean:
    if col in genres:
        movies_1975_bool[col] = movies_1975_clean[col].astype(bool)
    else:
        movies_1975_bool[col] = movies_1975_clean[col]

display(movies_1975_bool.head(4))

## Transformación de variables (calidad de datos)
- Valores no definidos
- Valores duplicados

In [ ]:
matrix = pd.DataFrame(np.random.randint(10,size=(5,10)).astype(float))
matrix[matrix < 2] = np.nan
display(matrix)

In [ ]:
# isnull() e isna() son equivalentes

na_col = matrix.isnull().sum()
na_col.name = "Nulos por columna"
print(na_col)
print()

na_row = matrix.isna().sum(axis=1)
na_row.name = "Nulos por fila"
print(na_row)

In [ ]:
# Cantidad valores nulos
print(matrix.isnull().sum(axis = 0).sum())

In [ ]:
# numero de no nulos
na_col = matrix.count()
na_col.name = "Valores válidos por columna"
print(na_col)
print()

na_row = matrix.count(axis=1)
na_row.name = "Valores válidos por fila"
print(na_row)

In [ ]:
# Representación de las filas en las que una determinada columna tiene nulos
display(matrix[matrix[3].isnull()])

In [ ]:
# Conteo de valores que aparecen en el dataset
valores = [2, 8]
# Identificación de valores de dominio que se encuentran en un listado
display(matrix[matrix[3].isin(valores)])

In [ ]:
display(matrix)

In [ ]:
## Tratamiento de valores nulos
print("Eliminar filas con NaN")
display(matrix.dropna(axis=0))

print("Eliminar columnas con NaN")
display(matrix.dropna(axis=1))

In [ ]:
# eliminar filas donde no hay al menos un número de valores no NaN
display(matrix)
display(matrix.dropna(thresh=8, axis=0))

In [ ]:
# sustituir por un valor fijo
display(matrix.fillna(-1))

In [ ]:
# sustituir por valor dinámico (copia)
display(matrix)

# display(matrix.bfill())
display(matrix.ffill())

In [ ]:
# sustituir por valor dinámico (interpolación)
display(matrix)
display(matrix.interpolate())

#### Tratar valores duplicados

In [ ]:
serie = pd.Series(['a','b','c','a','c','a','g'])
print(serie.duplicated())

In [ ]:
df = all_dataset.copy()
display(df.head(3))

# Eliminación de los duplicados en una columna definida
df2 = df.drop_duplicates(subset="Title", keep='last', inplace=False)
display(df2.head(2))

## Valores categóricos

In [ ]:
print(df.info())

In [ ]:
df["Age"] = pd.Categorical(df["Age"])

df.info()

In [ ]:
df["Rating"] = df["Rating"].astype("category")

df.info()

#### Discretización (valores categóricos)
- Tras Series y DataFrame, objeto para categorías: Categorical
```python
categorias = pd.cut(<valores>, <bins>) 
```

In [ ]:
# especificar los bloques
bins = [0,18,35,65,99, np.inf] # Límites para definir 5 categorías
etiquetas = ["Menor","Joven","Adulto","Mayor","Anciano"]
edades = [16,25,18,71,44,100,12]

categorias = pd.cut(edades,bins,labels=etiquetas)
display(categorias)
print()
print("Verificar si pertenece a una categoría:")
display(categorias == "Menor")

In [ ]:
# Cuantos por categoría?
categorias.value_counts()

In [ ]:
# Categorías por rango de valores
bins = 3
etiquetas = ["Eliminado", "Repechaje", "Clasificado"]
puntos = [0,6,8,16,25,18,71,44,100]

# Utilizar cut me devuelve rangos del mismo tamaño (max - min)/bin
categorias = pd.cut(puntos, bins, labels=etiquetas) 
print(categorias)
print()
print(categorias.value_counts())

In [ ]:
bins = 3
etiquetas = ["Eliminado", "Repechaje", "Clasificado"]
edades = [1,6,8,16,25,18,71,44,100]

# Utilizar qcut me devuelve distribuciones del mismo tamaño
categorias = pd.qcut(edades,bins,labels=etiquetas) # rangos homogéneos (similar número de valores)
print(categorias)
print()
print(categorias.value_counts())